In [12]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
csv_file = pd.read_csv(f"{location}/ERC_correlation.csv")

csv_file.index = csv_file.columns
string_to_remove = "_nostop.fas.aligned.fas.best"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

string_to_remove = ".fas.ClipKIT"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


matrix_long = csv_file.reset_index().melt(
    id_vars="index",         # Keeps row names as a column
    var_name="Gene1",       # New name for columns
    value_name="Gene2"       # New name for matrix values
)

# Rename 'index' to 'Row' for clarity
diapause = matrix_long.rename(columns={"index": "Row"})

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"
# location = "/mnt/c/Users/Saurav Baral/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)



genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.1]
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK"]
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation

gene_dict = gene_dictionary

In [2]:
import random
gene_dict = dict(random.sample(list(gene_dictionary.items()), 1000))

import networkx as nx

# Create a graph
G = nx.Graph()

# Add edges with similarity weights
for gene, neighbors in gene_dict.items():
    for neighbor, similarity in neighbors.items():
        G.add_edge(gene, neighbor, weight=similarity)


import numpy as np

# List of genes
genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))

# Fill the similarity matrix
for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = gene_dict[gene1][gene2]
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)

print("Similarity Matrix:")
print(np.shape(similarity_matrix))

Similarity Matrix:
(1000, 1000)


In [34]:
import multiprocessing as mp

def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the average similarity within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Compute the average similarity (excluding diagonal)
    return np.sum(submatrix) / (len(cluster) * (len(cluster) - 1))


def find_best_pair(genes, similarity_matrix, remaining_genes):
    """
    Helper function to find the most similar pair of genes in parallel.
    """
    best_pair = None
    best_similarity = -1
    
    for i, gene1 in enumerate(remaining_genes):
        for j, gene2 in enumerate(remaining_genes):
            if i >= j:
                continue  # Avoid duplicate pairs and self-pairs
            sim = similarity_matrix[genes.index(gene1)][genes.index(gene2)]
            if sim > best_similarity:
                best_similarity = sim
                best_pair = (gene1, gene2)
    
    return best_pair, best_similarity

def greedy_clustering_parallel(similarity_matrix, genes, max_clusters=2, num_cores=8, threshold=0.45, verbose=False):
    clusters = []
    remaining_genes = set(genes)
    
    while remaining_genes:
        chunk_size = len(remaining_genes) // num_cores
        chunks = [list(remaining_genes)[i:i + chunk_size] for i in range(0, len(remaining_genes), chunk_size)]
        
        with mp.Pool(processes=num_cores) as pool:
            results = pool.starmap(
                find_best_pair,
                [(genes, similarity_matrix, chunk) for chunk in chunks]
            )
        
        best_pair, best_similarity = None, -1
        for pair, similarity in results:
            if similarity > best_similarity:
                best_pair, best_similarity = pair, similarity
        
        if not best_pair:
            break
        
        cluster = set(best_pair)
        remaining_genes -= cluster
        
        for gene in list(remaining_genes):
            candidate_cluster = cluster | {gene}
            new_avg_sim = average_similarity(candidate_cluster, similarity_matrix, genes)
            if new_avg_sim > threshold:
                cluster.add(gene)
                remaining_genes.remove(gene)
        
        if cluster:
            clusters.append(cluster)
        
        if verbose:
            print(f"Formed Cluster {len(clusters)}: {cluster}, Remaining Genes: {len(remaining_genes)}")
        
        if len(clusters) >= max_clusters and not remaining_genes:
            break
    
    return clusters	
	

	

In [11]:
if __name__ == "__main__":  # Required for multiprocessing on Windows
    # Run the parallel greedy clustering algorithm
    # for i in range(20,55,5):
    
    
    i = 20
    print(i)
    clusters = greedy_clustering_parallel(similarity_matrix, genes, max_clusters=i, num_cores=16,threshold=0.40,min_cluster_size=4,verbose=True)
    print(len(clusters))
# Print the clusters
    for i, cluster in enumerate(clusters):
        if "period" in cluster:
            print(f"Cluster {i}: {cluster}")

20
Formed Cluster 1: {'SMARCA4', 'PAPD5', 'TDO2', 'SLCO4A1', 'CCT3', 'NOVA1', 'AP1B1', '5864at7088', 'WDR44', 'DDX56', 'GPAM', 'ATP5F1', 'MAP4K1', 'FOXK2', '4545at7088', '4485at7088', 'SLC26A2', 'GMPS', 'STK17B', 'DDIT4L', 'VAV2', '8698at7088', 'CRKL', 'cry1a', 'POLE', 'MTMR12', 'HMX3', 'MIF4GD', 'XPO5', 'RNF20', 'ATAD1', 'LONP1', 'SEC23A', 'NCBP1', 'MFN2', 'PIK3R4', 'SURF4', 'DARS', 'PNPLA6', 'Hil', 'SPPL3', 'MTHFSD', 'UBE3B', 'CNOT6L', 'CDC40', 'VPS28', 'AGPS', 'BRAF', 'SNW1', 'NOP56', '6279at7088', 'RCHY1', 'TKT', 'TGFB1I1', 'MAEA', 'CPSF2', 'CSTF3', '4238at7088', 'MAU2', 'AMOTL1', 'abs', 'ZPR1', 'ARFIP1', 'STXBP1', 'EEF2', 'CHKA', 'NTAN1', 'METTL14', 'EZH1', 'FRY', 'AGAP3', 'hrg', 'FBL', '1775at7088', 'PGM3', 'PDS5A', 'MICU3', 'EIF3A', 'PITX2', 'KDM2A', 'OPA1', 'NDUFV1', 'NUP93', 'HIP1', 'TTC14', 'ACTL6A', 'CdsA', 'SLU7', 'ESRRB', 'TRPM7', 'MARCH6', 'EIF5B', 'PANK1', 'slc25a4', '5925at7088', 'SNX27', 'FSCN1', 'AP1G1', 'CPSF6', '8841at7088', '2994at7088', 'PI4K2B', 'PSMA2', 'HNF4A',

Process ForkPoolWorker-9590:
Process ForkPoolWorker-9594:
Process ForkPoolWorker-9588:
Process ForkPoolWorker-9597:
Process ForkPoolWorker-9595:
Process ForkPoolWorker-9599:
Process ForkPoolWorker-9598:
Process ForkPoolWorker-9586:
Process ForkPoolWorker-9596:
Process ForkPoolWorker-9589:
Process ForkPoolWorker-9593:
Process ForkPoolWorker-9587:
Process ForkPoolWorker-9591:
Process ForkPoolWorker-9592:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-9600:
  File "/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/multiproc

In [9]:
def greedy_clustering_parallel(similarity_matrix, genes, max_clusters=2, num_cores=8, threshold=0.45, min_cluster_size=4, verbose=False):
    clusters = []
    remaining_genes = set(genes)
    
    while remaining_genes:
        chunk_size = len(remaining_genes) // num_cores
        chunks = [list(remaining_genes)[i:i + chunk_size] for i in range(0, len(remaining_genes), chunk_size)]
        
        with mp.Pool(processes=num_cores) as pool:
            results = pool.starmap(
                find_best_pair,
                [(genes, similarity_matrix, chunk) for chunk in chunks]
            )
        
        best_pair, best_similarity = None, -1
        for pair, similarity in results:
            if similarity > best_similarity:
                best_pair, best_similarity = pair, similarity
        
        if not best_pair:
            break
        
        cluster = set(best_pair)
        remaining_genes -= cluster
        
        for gene in list(remaining_genes):
            candidate_cluster = cluster | {gene}
            new_avg_sim = average_similarity(candidate_cluster, similarity_matrix, genes)
            if new_avg_sim > threshold:
                cluster.add(gene)
                remaining_genes.remove(gene)
        
        # Ensure the cluster meets the minimum size
        if len(cluster) >= min_cluster_size:
            clusters.append(cluster)
        
        if verbose:
            print(f"Formed Cluster {len(clusters)}: {cluster}, Remaining Genes: {len(remaining_genes)}")
        
        if len(clusters) >= max_clusters and not remaining_genes:
            break
    
    return clusters

In [13]:
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform
import numpy as np

# Ensure symmetry of the similarity matrix
distance_matrix = 1 - similarity_matrix
distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Symmetrize
np.fill_diagonal(distance_matrix, 0)  # Set diagonal to zero

# Convert to condensed form
condensed_distance_matrix = squareform(distance_matrix)

# Perform hierarchical clustering
Z = linkage(condensed_distance_matrix, method='average')

# Extract clusters based on a distance threshold
distance_threshold = 0.55
clusters = fcluster(Z, t=distance_threshold, criterion='distance')

# Map genes to clusters
from collections import defaultdict
gene_clusters = defaultdict(list)
for gene, cluster_id in zip(genes, clusters):
    gene_clusters[cluster_id].append(gene)

# Filter clusters with at least 4 members
min_cluster_size = 4
filtered_clusters = {
    cluster_id: cluster_genes
    for cluster_id, cluster_genes in gene_clusters.items()
    if len(cluster_genes) >= min_cluster_size
}

# Print filtered clusters
for cluster_id, cluster_genes in filtered_clusters.items():
    print(f"Cluster {cluster_id} (size {len(cluster_genes)}): {cluster_genes}")

ValueError: Distance matrix 'X' must be symmetric.

In [6]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform
from collections import defaultdict
import matplotlib.pyplot as plt

# Example input: Replace these with your actual data
# genes = ["gene1", "gene2", "gene3", "gene4", "gene5", "gene6"]
# similarity_matrix = np.array([
#     [1.0, 0.8, 0.6, 0.2, 0.1, 0.4],
#     [0.8, 1.0, 0.7, 0.3, 0.2, 0.5],
#     [0.6, 0.7, 1.0, 0.4, 0.3, 0.6],
#     [0.2, 0.3, 0.4, 1.0, 0.8, 0.7],
#     [0.1, 0.2, 0.3, 0.8, 1.0, 0.9],
#     [0.4, 0.5, 0.6, 0.7, 0.9, 1.0],
# ])

# Step 1: Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix  # Distance = 1 - similarity
print(np.shape(distance_matrix))

threshold = 1e-10

# Round small values in the matrix to zero
distance_matrix[np.abs(distance_matrix) < threshold] = 0

# Ensure the matrix is symmetric and the diagonal is 0
distance_matrix = (distance_matrix + distance_matrix.T) / 2
np.fill_diagonal(distance_matrix, 0)

# Convert to condensed form for hierarchical clustering
condensed_distance_matrix = squareform(distance_matrix)

# Step 2: Perform hierarchical clustering
# You can use 'average', 'single', 'complete', or 'ward' for linkage method
Z = linkage(condensed_distance_matrix, method='average')

# Step 3: Extract clusters
# Set a distance threshold or specify the number of clusters
distance_threshold = 0.55
clusters = fcluster(Z, t=distance_threshold, criterion='distance')

# Map genes to their clusters
gene_clusters = defaultdict(list)
for gene, cluster_id in zip(genes, clusters):
    gene_clusters[cluster_id].append(gene)

# Step 4: Filter clusters with a minimum size of 4
min_cluster_size = 2
filtered_clusters = {
    cluster_id: cluster_genes
    for cluster_id, cluster_genes in gene_clusters.items()
    if len(cluster_genes) >= min_cluster_size
}

# Step 5: Print clusters
print("Clusters with at least 4 members:")
for cluster_id, cluster_genes in filtered_clusters.items():
    print(f"Cluster {cluster_id} (size {len(cluster_genes)}): {cluster_genes}")


(1000, 1000)


ValueError: Distance matrix 'X' must be symmetric.

In [28]:
import networkx as nx
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform

import random
# gene_dict = dict(random.sample(list(gene_dictionary.items()), 1000))
gene_dict = gene_dictionary

# Create a graph
G = nx.Graph()

# Add edges with similarity weights
for gene, neighbors in gene_dict.items():
    for neighbor, similarity in neighbors.items():
        G.add_edge(gene, neighbor, weight=similarity)

# List of genes
genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))



In [29]:
# Fill the similarity matrix and ensure symmetry
for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = round(gene_dict[gene1][gene2],2)
            similarity_matrix[j, i] = round(gene_dict[gene1][gene2],2)  # Ensure symmetry
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)


min_similarity = np.min(similarity_matrix)
# print(similarity_matrix)
# if min_similarity < 0:
#     similarity_matrix = similarity_matrix - min_similarity  # Shift to make all values non-negative
    
# Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

np.fill_diagonal(distance_matrix, 0)

# Flatten the distance matrix to a condensed form (required by scipy)
condensed_distance_matrix = squareform(distance_matrix)

# Perform hierarchical clustering
# n_clusters can be set to the desired number of clusters
# Alternatively, you can use distance_threshold to cut the dendrogram


In [44]:
import statistics

for i in range(1,3000,200):
    n_clusters = i  # Adjust this based on your needs
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        affinity="precomputed",  # Use precomputed distances
        linkage="average"  # You can change this to "single", "complete", or "ward"
    )
    
    # Fit the model
    clustering.fit(distance_matrix)
    
    # Get the cluster labels
    labels = clustering.labels_
    
    # Organize genes into clusters
    clusters = {}
    for gene, label in zip(genes, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(gene)
    
    # Print the clusters
    similarity_list = []
    for cluster_id, genes_in_cluster in clusters.items():
        similarity_list.append(average_similarity(genes_in_cluster, similarity_matrix, genes))
        # if "period" in genes_in_cluster:
        #     print(f"Cluster {cluster_id}: {genes_in_cluster}, {average_similarity(genes_in_cluster, similarity_matrix, genes)}")
    
    print(i,statistics.mean(similarity_list),statistics.median(similarity_list))


/home/saurav/.local/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


1 0.08098742500807968 0.08098742500807968


/home/saurav/.local/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


201 0.23351353051080953 0.23044444444444448


/home/saurav/.local/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


...

2401 0.19157988391075004 0.0


/home/saurav/.local/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


2601 0.1752409870990018 0.0


/home/saurav/.local/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


2801 0.15665672380178497 0.0
